In [38]:
import pandas as pd

In [39]:
#Find your user_id by logging into your account, going to your profile, going to your workouts, and get the link from "Download Workout"
import requests
from config import user_id, username, password
s = requests.Session()
payload = {'username_or_email':username, 'password':password}
s.post('https://api.onepeloton.com/auth/login', json=payload)
download = f'https://api.onepeloton.com/api/user/{user_id}/workout_history_csv?timezone=America/Chicago'
response = s.get(download)
with open('C:/Users/erdos/Class Folder/Peloton_Data/Resources/NickNaught_workouts.csv', 'wb') as file:
    file.write(response.content)

In [ ]:
#read the imported CSV file into a dataframe
df = pd.read_csv('Resources/NickNaught_workouts.csv')

In [45]:
# convert the column of time objects to datetime objects
df['date'] = df['Workout Timestamp'].str.slice(stop=-12)
df['Workout Timestamp'] = df['date'].apply(pd.to_datetime)
df.drop(['date','Class Timestamp','Avg. Incline','Avg. Pace (min/mi)'], axis=1, inplace=True)
df.fillna(0, inplace=True)
#Adding a seconds column that will be later used to calculate a TSS (Training Stress Score)
df['Length (seconds)'] = df['Length (minutes)']*60
df

,Workout Timestamp,Live/On-Demand,Instructor Name,Length (minutes),Fitness Discipline,Type,Title,Total Output,Avg. Watts,Avg. Resistance,Avg. Cadence (RPM),Avg. Speed (mph),Distance (mi),Calories Burned,Avg. Heartrate,Length (seconds)
0,2023-01-30,On Demand,Ben Alldis,5,Cycling,Warm Up/Cool Down,5 min Warm Up Ride,41.0,137.0,39%,90.0,17.95,1.49,61,0.00,300
1,2023-01-31,Live,Jenn Sherman,15,Cycling,Intervals,15 min HIIT Ride,137.0,153.0,41%,84.0,17.95,4.49,256,159.52,900
2,2023-02-01,Live,Robin Arzon,15,Cycling,Music,15 min 80s Ride,112.0,125.0,41%,84.0,17.55,4.39,206,137.85,900
3,2023-02-02,Live,Bradley Rose,30,Cycling,Low Impact,30 min Low Impact Ride,150.0,84.0,35%,78.0,14.61,7.30,322,118.81,1800
4,2023-02-04,On Demand,Bradley Rose,15,Cycling,Climb,15 min Climb Ride,158.0,176.0,51%,72.0,19.63,4.91,253,157.94,900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2023-03-25,On Demand,Ben Alldis,60,Cycling,Power Zone,60 min Power Zone Ride,581.0,158.0,43%,86.0,19.16,19.37,903,144.73,3600
59,2023-03-25,On Demand,Bradley Rose,5,Stretching,Pre & Post-Ride Stretch,5 min Post-Ride Stretch,0.0,0.0,0,0.0,0.00,0.00,68,136.75,300
60,2023-03-27,On Demand,Christine D'Ercole,45,Cycling,Power Zone,45 min Power Zone Endurance Ride,383.0,142.0,42%,86.0,18.42,13.82,577,132.06,2700
61,2023-03-27,On Demand,Sam Yo,5,Cycling,Warm Up/Cool Down,5 min Cool Down Ride,29.0,98.0,39%,76.0,15.62,1.29,64,132.97,300


In [47]:
#Setup initial FTP score: 
# set the threshold date before which we want to add the FTP value
threshold_date = pd.to_datetime('2023-04-01')

# add a new column 'Current FTP' with a default value of 0
df['Current FTP'] = 0

# set the value of 'Current FTP' to 200 for the rows where the date is before the threshold date
df.loc[df['Workout Timestamp'] < threshold_date, 'Current FTP'] = 200

In [56]:
#move the columns to a place that makes sense. 
seconds_column = df.pop('Length (seconds)')
df.insert(4, 'Length (seconds)', seconds_column)

ftp_column = df.pop('Current FTP')
df.insert(8, 'Current FTP', ftp_column)

df

,Workout Timestamp,Live/On-Demand,Instructor Name,Length (minutes),Length (seconds),Fitness Discipline,Type,Title,Current FTP,Total Output,Avg. Watts,Avg. Resistance,Avg. Cadence (RPM),Avg. Speed (mph),Distance (mi),Calories Burned,Avg. Heartrate
0,2023-01-30,On Demand,Ben Alldis,5,300,Cycling,Warm Up/Cool Down,5 min Warm Up Ride,200,41.0,137.0,39%,90.0,17.95,1.49,61,0.00
1,2023-01-31,Live,Jenn Sherman,15,900,Cycling,Intervals,15 min HIIT Ride,200,137.0,153.0,41%,84.0,17.95,4.49,256,159.52
2,2023-02-01,Live,Robin Arzon,15,900,Cycling,Music,15 min 80s Ride,200,112.0,125.0,41%,84.0,17.55,4.39,206,137.85
3,2023-02-02,Live,Bradley Rose,30,1800,Cycling,Low Impact,30 min Low Impact Ride,200,150.0,84.0,35%,78.0,14.61,7.30,322,118.81
4,2023-02-04,On Demand,Bradley Rose,15,900,Cycling,Climb,15 min Climb Ride,200,158.0,176.0,51%,72.0,19.63,4.91,253,157.94
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2023-03-25,On Demand,Ben Alldis,60,3600,Cycling,Power Zone,60 min Power Zone Ride,200,581.0,158.0,43%,86.0,19.16,19.37,903,144.73
59,2023-03-25,On Demand,Bradley Rose,5,300,Stretching,Pre & Post-Ride Stretch,5 min Post-Ride Stretch,200,0.0,0.0,0,0.0,0.00,0.00,68,136.75
60,2023-03-27,On Demand,Christine D'Ercole,45,2700,Cycling,Power Zone,45 min Power Zone Endurance Ride,200,383.0,142.0,42%,86.0,18.42,13.82,577,132.06
61,2023-03-27,On Demand,Sam Yo,5,300,Cycling,Warm Up/Cool Down,5 min Cool Down Ride,200,29.0,98.0,39%,76.0,15.62,1.29,64,132.97


Only run code when the FTP score needs to change

In [ ]:
def add_current_ftp(df, new_ftp, date):
    # set the value of 'Current FTP' to the input value for the rows where the date is after the input date
    df.loc[df['Workout Timestamp'] >= date, 'Current FTP'] = new_ftp
    
    # print the resulting DataFrame
    print(df)

#add the date the new FTP score was documented. 
add_current_ftp(df, <new ftp value>, pd.to_datetime(<date of new ftp>))

In [62]:
df['Fitness Indicator'] = df['Avg. Watts']/df['Avg. Heartrate']
df

,Workout Timestamp,Live/On-Demand,Instructor Name,Length (minutes),Length (seconds),Fitness Discipline,Type,Title,Current FTP,Total Output,Avg. Watts,Avg. Resistance,Avg. Cadence (RPM),Avg. Speed (mph),Distance (mi),Calories Burned,Avg. Heartrate,Fitness Indicator
0,2023-01-30,On Demand,Ben Alldis,5,300,Cycling,Warm Up/Cool Down,5 min Warm Up Ride,200,41.0,137.0,39%,90.0,17.95,1.49,61,0.00,inf
1,2023-01-31,Live,Jenn Sherman,15,900,Cycling,Intervals,15 min HIIT Ride,200,137.0,153.0,41%,84.0,17.95,4.49,256,159.52,0.959127
2,2023-02-01,Live,Robin Arzon,15,900,Cycling,Music,15 min 80s Ride,200,112.0,125.0,41%,84.0,17.55,4.39,206,137.85,0.906783
3,2023-02-02,Live,Bradley Rose,30,1800,Cycling,Low Impact,30 min Low Impact Ride,200,150.0,84.0,35%,78.0,14.61,7.30,322,118.81,0.707011
4,2023-02-04,On Demand,Bradley Rose,15,900,Cycling,Climb,15 min Climb Ride,200,158.0,176.0,51%,72.0,19.63,4.91,253,157.94,1.114347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,2023-03-25,On Demand,Ben Alldis,60,3600,Cycling,Power Zone,60 min Power Zone Ride,200,581.0,158.0,43%,86.0,19.16,19.37,903,144.73,1.091688
59,2023-03-25,On Demand,Bradley Rose,5,300,Stretching,Pre & Post-Ride Stretch,5 min Post-Ride Stretch,200,0.0,0.0,0,0.0,0.00,0.00,68,136.75,0.000000
60,2023-03-27,On Demand,Christine D'Ercole,45,2700,Cycling,Power Zone,45 min Power Zone Endurance Ride,200,383.0,142.0,42%,86.0,18.42,13.82,577,132.06,1.075269
61,2023-03-27,On Demand,Sam Yo,5,300,Cycling,Warm Up/Cool Down,5 min Cool Down Ride,200,29.0,98.0,39%,76.0,15.62,1.29,64,132.97,0.737008
